# Initial Groq + Citi Bike Setup

This notebook walks through connecting to the Groq LLM API and preparing the Citi Bike SQLite database so you can issue prompts and SQL queries.

In [ ]:
!wget -q https://raw.githubusercontent.com/alikarami76/genai-workshop/refs/heads/main/db_loader.py
!wget -q https://raw.githubusercontent.com/alikarami76/genai-workshop/refs/heads/main/llm_connector.py
!wget -q https://raw.githubusercontent.com/alikarami76/genai-workshop/refs/heads/main/requirements.txt

## Step 1. Install dependencies

Run the cell below once to install the `groq` client library. The command is idempotent but requires network access.

In [ ]:
%pip install -r requirements.txt

## Step 2. Configure the Groq client

Set your API key directly in this cell and instantiate the client. Replace the placeholder string with your actual key from the Groq console.

In [ ]:
from groq import Groq

# Replace with your actual Groq API key before running.
GROQ_API_KEY = "gsk_replace_with_your_actual_key"

client = Groq(api_key=GROQ_API_KEY)


def ask_groq(prompt: str, *, model: str = "qwen/qwen3-32b") -> str:
    """Send a user prompt to Groq and return the assistant's reply."""
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful data assistant."},
            {"role": "user", "content": prompt},
        ],
    )
    return response.choices[0].message.content.strip()


## Step 3. Prepare the Citi Bike database connection

The helper from `db_loader.py` downloads the dataset from Kaggle (if needed) and returns both a SQLite connection and a `run_query` convenience wrapper. Ensure your Kaggle credentials and CLI access are configured before running.

In [ ]:
from db_loader import prepare_citibike_database

sqlite_path, conn, run_query = prepare_citibike_database()
if run_query is None:
    raise RuntimeError("Query helper was not created. Re-run the setup cell with build_query_runner=True.")

print(f"Citi Bike database ready at: {sqlite_path}")
conn


## Step 4. Send a prompt to the Groq LLM

Use the helper to issue a prompt and capture the response. Adjust the prompt text as needed for your workflow.

In [ ]:
from IPython.display import Markdown, display
prompt = "How does an LLM work?"
llm_response = ask_groq(prompt)
display(Markdown(llm_response))

## Step 5. Inspect the database schema

Query `sqlite_master` to view every table definition in the database. This provides an overview of the available tables and columns.

In [ ]:
schema_query = """
SELECT name, sql
FROM sqlite_master
WHERE type = 'table'
ORDER BY name;
"""
schema_df = run_query(schema_query)
schema_df


## Step 6. Run a sample data query

Fetch a few rows from one of the tables to verify the connection and preview the data.

In [ ]:
if schema_df.empty:
    raise RuntimeError("No tables found in the Citi Bike database.")

sample_table = schema_df.loc[0, "name"]
table_identifier = sample_table.replace('\"', '\"\"')
print(f"Previewing data from table: {sample_table}")
sample_df = run_query(f'SELECT * FROM "{table_identifier}" LIMIT 5;')
sample_df
